#### Multi-class, top-k prediction task

Overview

Welcome to the 2025 Kaggle Playground Series! We plan to continue in the spirit of previous playgrounds, providing interesting and approachable datasets for our community to practice their machine learning skills, and anticipate a competition each month.

Your Goal: Your objective is to select the best fertilizer for different weather, soil conditions and crops.

Dataset Description

The dataset for this competition (both train and test) was generated from a deep learning model trained on the Fertilizer prediction dataset. Feature distributions are close to, but not exactly the same, as the original. Feel free to use the original dataset as part of this competition, both to explore differences as well as to see whether incorporating the original in training improves model performance.

Files

* train.csv - the training dataset; Fertilizer Name is the categorical target
* test.csv - the test dataset; your objective is to predict the Fertilizer Name for each row, up to three value, space delimited.
* sample_submission.csv - a sample submission file in the correct format.

Submission File

For each id in the test set, you may predict up to 3 Fertilizer Name values, with the predictions space delimited. The file should contain a header and have the following format:

id,Fertilizer Name 

750000,14-35-14 10-26-26 Urea

750000,14-35-14 10-26-26 Urea 

...

In [50]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.cluster import KMeans

from xgboost import XGBClassifier

In [51]:
df = pd.read_csv("../Data/train.csv")
X_test = pd.read_csv("../Data/test.csv")
test_ids = X_test[["id"]]

In [52]:
# Feature engineering
def feat_eng(df):
    df['N_to_P'] = df['Nitrogen'] / df['Phosphorous'].replace(0, 0.1)
    df['N_to_K'] = df['Nitrogen'] / df['Potassium'].replace(0, 0.1)
    df['P_to_K'] = df['Phosphorous'] / df['Potassium'].replace(0, 0.1)
    df['NPK_sum'] = df['Nitrogen'] + df['Phosphorous'] + df['Potassium']
    df['SoilCropCombo'] = df['Soil Type'] + '_' + df['Crop Type']

    # cluster N, P, K combinations
    kmeans = KMeans(n_clusters=5, random_state=42)
    df['NutrientCluster'] = kmeans.fit_predict(df[['Nitrogen', 'Potassium', 'Phosphorous']])

    # create interaction terms
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
    poly_cols = ['Nitrogen', 'Potassium', 'Phosphorous']
    poly_feats = poly.fit_transform(df[['Nitrogen', 'Potassium', 'Phosphorous']])
    poly_feats_names = poly.get_feature_names_out(poly_cols)
    df_poly_feats = pd.DataFrame(poly_feats, columns=poly_feats_names)
    df = df.drop(columns=poly_cols)
    df = pd.merge(df, df_poly_feats, left_index=True, right_index=True)

    return df

df = feat_eng(df)
X_test = feat_eng(X_test)

In [53]:
y = df["Fertilizer Name"]
X = df.drop(columns=["id", "Fertilizer Name", 'Soil Type', 'Crop Type'])
X_test = X_test.drop(columns=["id", 'Soil Type', 'Crop Type'])
X.head()

,Temparature,Humidity,Moisture,N_to_P,N_to_K,P_to_K,NPK_sum,SoilCropCombo,NutrientCluster,Nitrogen,Potassium,Phosphorous,Nitrogen Potassium,Nitrogen Phosphorous,Potassium Phosphorous
0,37,70,36,7.200000,9.00,1.250000,45,Clayey_Sugarcane,2,36.0,4.0,5.0,144.0,180.0,20.0
1,27,69,65,1.666667,5.00,3.000000,54,Sandy_Millets,1,30.0,6.0,18.0,180.0,540.0,108.0
2,29,63,32,1.500000,2.00,1.333333,52,Sandy_Millets,1,24.0,12.0,16.0,288.0,384.0,192.0
3,35,62,54,9.750000,3.25,0.333333,55,Sandy_Barley,2,39.0,12.0,4.0,468.0,156.0,48.0
4,35,58,43,2.312500,18.50,8.000000,55,Red_Paddy,2,37.0,2.0,16.0,74.0,592.0,32.0


In [54]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
X_encoded = pd.get_dummies(X, columns=['SoilCropCombo'], prefix='', prefix_sep='').astype(int)
X_test = pd.get_dummies(X_test, columns=['SoilCropCombo'], prefix='', prefix_sep='').astype(int)
X_encoded.head()

,Temparature,Humidity,Moisture,N_to_P,N_to_K,P_to_K,NPK_sum,NutrientCluster,Nitrogen,Potassium,...,Sandy_Cotton,Sandy_Ground Nuts,Sandy_Maize,Sandy_Millets,Sandy_Oil seeds,Sandy_Paddy,Sandy_Pulses,Sandy_Sugarcane,Sandy_Tobacco,Sandy_Wheat
0,37,70,36,7,9,1,45,2,36,4,...,0,0,0,0,0,0,0,0,0,0
1,27,69,65,1,5,3,54,1,30,6,...,0,0,0,1,0,0,0,0,0,0
2,29,63,32,1,2,1,52,1,24,12,...,0,0,0,1,0,0,0,0,0,0
3,35,62,54,9,3,0,55,2,39,12,...,0,0,0,0,0,0,0,0,0,0
4,35,58,43,2,18,8,55,2,37,2,...,0,0,0,0,0,0,0,0,0,0


In [55]:
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y_encoded, test_size=0.4, random_state=42)

In [56]:
# function for MAP@3
def mapk(actual, predicted, k=3):
    score = 0.0
    for a, p in zip(actual, predicted):
        if a in p[:k]:
            index = p.index(a)
            score += 1.0 / (index + 1)
    return score / len(actual)

In [58]:
params = {
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.3],
}

xgb = XGBClassifier(eval_metric='mlogloss')
search = RandomizedSearchCV(xgb, params, scoring='neg_log_loss', cv=3, n_iter=20, verbose=4)
search.fit(X_encoded, y_encoded)
xgb_model = search.best_estimator_ 

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END colsample_bytree=0.8, gamma=0.3, learning_rate=0.1, max_depth=5, n_estimators=200, subsample=0.6;, score=-1.928 total time=  16.4s
[CV 2/3] END colsample_bytree=0.8, gamma=0.3, learning_rate=0.1, max_depth=5, n_estimators=200, subsample=0.6;, score=-1.929 total time=  16.9s
[CV 3/3] END colsample_bytree=0.8, gamma=0.3, learning_rate=0.1, max_depth=5, n_estimators=200, subsample=0.6;, score=-1.928 total time=  18.3s
[CV 1/3] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=10, n_estimators=100, subsample=0.8;, score=-1.931 total time=  13.9s
[CV 2/3] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=10, n_estimators=100, subsample=0.8;, score=-1.931 total time=  13.6s
[CV 3/3] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=10, n_estimators=100, subsample=0.8;, score=-1.930 total time=  12.2s
[CV 1/3] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.01, max

In [59]:
xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=0.1,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
xgb_probs = xgb_model.predict_proba(X_val)
xgb_top3 = np.argsort(xgb_probs, axis=1)[:, -3:][:, ::-1]
xgb_labels = [[encoder.classes_[i] for i in row] for row in xgb_top3]
actual_labels = [encoder.classes_[i] for i in y_val]
print("XGBoost MAP@3:", mapk(actual_labels, xgb_labels))

# run 1: XGBoost MAP@3: 0.31670999999987165

d:\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:10:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost MAP@3: 0.31670999999987165


In [ ]:
y_probs = xgb_model.predict_proba(X_test)
top_3_preds = np.argsort(y_probs, axis=1)[:, -3:][:, ::-1]
X_test_result = X_test.copy().reset_index(drop=True)
top_3_labels = [[encoder.classes_[idx] for idx in row] for row in top_3_preds]
top_3_str = [' '.join(preds) for preds in top_3_labels]
X_test_result['Fertilizer Name'] = top_3_str

test_final = pd.merge(test_ids, X_test_result[["Fertilizer Name"]], left_index=True, right_index=True).set_index("id")
test_final.to_csv("prediction_xgb_model_ext.csv")
test_final

,Fertilizer Name
id,
750000,28-28 DAP 10-26-26
750001,17-17-17 10-26-26 20-20
750002,10-26-26 28-28 20-20
750003,14-35-14 Urea 17-17-17
750004,20-20 10-26-26 28-28
...,...
999995,20-20 10-26-26 17-17-17
999996,14-35-14 10-26-26 28-28
999997,14-35-14 28-28 10-26-26
